In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import numpy as np
from pylab import rcParams

%matplotlib inline

In [2]:
# things we need for NLP
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

# things we need for Tensorflow
import numpy as np
import tflearn
import tensorflow as tf
import random
from gensim.models import Word2Vec

/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
import sys
import math
import tflearn
import tensorflow as tf
from tensorflow.python.ops import rnn_cell
from tensorflow.python.ops import rnn
import chardet
import numpy as np
import struct

In [4]:
# import our intents file
import json

def getdata(path):
    g = open(path,'r')
    for l in g:
        yield eval(l)


intents = list(getdata('/data/QA/QA_Video_Games.json'))

In [5]:
stop_words = nltk.corpus.stopwords.words('english') + [
    '.',
    ',',
    '--',
    '\'s',
    '?',
    ')',
    '(',
    ':',
    '\'',
    '\'re',
    '"',
    '-',
    '}',
    '{',
    u'—',
    '',
    ]

In [6]:
import string
import unicodedata
def CleanLines(text):
    cleanLine = []
    identify = string.maketrans('', '')
    delEStr = string.punctuation +string.digits
    for i in text:         
        i = i.encode("utf-8")
        lines = i.translate(identify,delEStr)
        cleanLine.append(lines) 
    return cleanLine

In [9]:
def uni(text):
    uni_text = []
    for i in text:
        i = unicode(i, "utf-8")
        uni_text.append(i)
    return uni_text

In [10]:
def deuni(text):
    deuni_text = []
    for i in text:
        i = i.encode("utf-8")
        deuni_text.append(i)
    return deuni_text

In [11]:
# delete long words
def long_word_filter(words):
    word_list = []
    for i in words:
        if len(i)<15:
            word_list.append(i)
        
    return word_list

In [12]:
words = []
classes = []
documents = []
answers = [] ##是question-answer pair

# loop through each question in our intents
for intent in intents:
    for question in intent['questions']:
        # tokenize each word in the sentence
        #text = CleanLines(question['questionText'])
        w = nltk.word_tokenize(question['questionText'])
        w = CleanLines(w)
        w = uni(w)
        # add to our words list
        words.extend(w)
        documents.append((w, intent['asin']))
        # add to our classes list
        if intent['asin'] not in classes:
            classes.append(intent['asin'])
        
        for answer in question['answers']:
            w2 = nltk.word_tokenize(answer['answerText'])
            w2 = CleanLines(w2)
            w2 = uni(w2)
            words.extend(w2)
            answers.append((w,w2))

# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in stop_words]
words = sorted(list(set(words)))
words = deuni(words)
words = long_word_filter(words)

# remove duplicates
classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes")
print (len(words), "unique stemmed words")
print (len(answers), "answers")

(7744, 'documents')
(1183, 'classes')
(13772, 'unique stemmed words')
(28893, 'answers')


In [13]:
model = Word2Vec(words, sg=1, size=100,  window=5,  min_count=5,  negative=3, sample=0.001, hs=1, workers=5)

In [14]:
model.save('/data/QA/Game.bin')

In [15]:
question_seqs = []
answer_seqs = []

In [16]:
# 有的参数没用
max_w = 50
float_size = 4
word_vector_dict = {}
word_vec_dim = 100
max_seq_len = 8
word_set = {}

In [17]:
input_file = open('/data/QA/Game.bin', "rb")

In [18]:
# answers是question-answer pair
questions = []
for i in range(len(answers)):
    question = deuni(answers[i][0])
    questions.append(question) 

In [19]:
#answer1是 单纯answer
answers1 =[]
for i in range(len(answers)):
    answer = deuni(answers[i][1])
    answers1.append(answer)

In [23]:
fo = open("/data/QA/QA_pair.txt", "w")

In [21]:
# version 1- all quesitons
for i in range(len(questions)):
    for w_q in questions[i]:
        w_q = w_q.lower()
        fo.write( w_q+' ')
        
    fo.write('|')
    
    for w_a in answers1[i]:
        w_a = w_a.lower()
        fo.write(w_a+' ')
    
    fo.write('\n')
            
fo.close()

In [24]:
# version 1-  1/10 of all quesitons
for i in range(int(len(questions)/15)):
    for w_q in questions[i]:
        w_q = w_q.lower()
        fo.write( w_q+' ')
        
    fo.write('|')
    
    for w_a in answers1[i]:
        w_a = w_a.lower()
        fo.write(w_a+' ')
    
    fo.write('\n')
            
fo.close()
    

In [11]:
fo = open("/data/QA/QA_pair.txt", "r")
print  len(fo.readlines())

1926


In [12]:
#load vector
model_w2v = Word2Vec.load('/data/QA/Game.bin')
word_vector=model_w2v.wv
#word_vocab = model_w2v.vocabulary
word_vector_dict = word_vector.vocab

In [13]:
def init_seq(input_file):
    """读取切好词的文本文件，加载全部词序列
    """
    file_object = open(input_file, 'r')
    vocab_dict = {}
    while True:
        question_seq = []
        answer_seq = []
        line = file_object.readline()
        if line:
            line_pair = line.split('|')
            line_question = line_pair[0]
            line_answer = line_pair[1]
            for word in line_question.split(' '):
                if word_vector_dict.has_key(word):
                    question_seq.append(word_vector[word])
            for word in line_answer.decode('utf-8').split(' '):
                if word_vector_dict.has_key(word):
                    answer_seq.append(word_vector[word])
        else:
            break
        question_seqs.append(question_seq)
        answer_seqs.append(answer_seq)
    file_object.close()

In [14]:
import sys
import math
import tflearn
import tensorflow as tf
from tensorflow.python.ops import rnn_cell
from tensorflow.python.ops import rnn
import chardet
import numpy as np
import struct

In [15]:
# get the vectorized question & answer 
question_seqs=[]
answer_seqs=[]
init_seq('/data/QA/QA_pair.txt')

In [16]:
type(question_seqs)

list

In [17]:
def get_max_seq_len(seqs):
    seq_lens = []
    for seq in seqs:
        seq_lens.append(len(seq))
        seq_lens.sort(reverse=True)
    return seq_lens[0]

In [18]:
max_q_seq = get_max_seq_len(question_seqs)
max_a_seq = get_max_seq_len(answer_seqs)
print max_q_seq,max_a_seq

47 76


In [19]:
max_seq_len = max_q_seq + max_a_seq
max_seq_len

123

In [56]:
class MySeq2Seq(object):
    def __init__(self, max_seq_len = 76, word_vec_dim = 100, input_file='/data/QA/QA_pair.txt'):
        self.max_seq_len = max_seq_len
        self.word_vec_dim = word_vec_dim
        self.input_file = input_file
    
    def generate_trainig_data(self):
 #       load_word_set()
 #       load_vectors("/data/QA/Game_words.bin")
        init_seq(self.input_file)
        xy_data = []
        y_data = []
        for i in range(len(question_seqs)):
        #for i in range(100):
            question_seq = question_seqs[i]
            answer_seq = answer_seqs[i]
            if len(question_seq) < self.max_seq_len and len(answer_seq) < self.max_seq_len:
                #多余的位设为0，与question的reverse合并，为什么要reverse？ * - repeat ；+ - 合并
                sequence_ry = [np.zeros(self.word_vec_dim)] * (self.max_seq_len-len(question_seq)) + list(reversed(question_seq))
                #多余的位设为0， 与answer合并
                sequence_y = answer_seq + [np.zeros(self.word_vec_dim)] * (self.max_seq_len-len(answer_seq))
                #合并
                sequence_xy = sequence_ry + sequence_y
                
                #sequence_xy = question_seq +[np.zeros(self.word_vec_dim)] * (self.max_seq_len-len(question_seq))
                
                sequence_y = [np.ones(self.word_vec_dim)] + sequence_y
                xy_data.append(sequence_xy)
                y_data.append(sequence_y)

                #print "right answer"
                #for w in answer_seq:
                #    (match_word, max_cos) = vector2word(w)
                #    if len(match_word)>0:
                #        print match_word, vector_sqrtlen(w)

        return np.array(xy_data), np.array(y_data)
    
    
    def model(self, feed_previous=False):
        # 通过输入的XY生成encoder_inputs和带GO头的decoder_inputs
        input_data = tflearn.input_data(shape=[None, self.max_seq_len*2, self.word_vec_dim], dtype=tf.float32, name = "XY")
        encoder_inputs = tf.slice(input_data, [0, 0, 0], [-1, self.max_seq_len, self.word_vec_dim], name="enc_in")
        decoder_inputs_tmp = tf.slice(input_data, [0, self.max_seq_len, 0], [-1, self.max_seq_len-1, self.word_vec_dim], name="dec_in_tmp")
        go_inputs = tf.ones_like(decoder_inputs_tmp)
        go_inputs = tf.slice(go_inputs, [0, 0, 0], [-1, 1, self.word_vec_dim])
                
        #decoder_inputs = tf.concat(1, [go_inputs, decoder_inputs_tmp], name="dec_in")
        decoder_inputs = go_inputs + decoder_inputs_tmp
        decoder_inputs = tf.slice(decoder_inputs,[0,0,0],[-1,-1,self.word_vec_dim],name="dec_in")


        
        
        # 编码器
        # 把encoder_inputs交给编码器，返回一个输出(预测序列的第一个值)和一个状态(传给解码器)
        with tf.variable_scope(tf.get_variable_scope(),reuse=tf.AUTO_REUSE):
            (encoder_output_tensor, states) = tflearn.lstm(encoder_inputs, self.word_vec_dim, return_state=True, scope='encoder_lstm')
            encoder_output_sequence = tf.stack([encoder_output_tensor], axis=1)
            

            # 解码器
            # 预测过程用前一个时间序的输出作为下一个时间序的输入
            # 先用编码器的最后一个输出作为第一个输入
            if feed_previous:
                first_dec_input = go_inputs
            else:
                first_dec_input = tf.slice(decoder_inputs, [0, 0, 0], [-1, 1, self.word_vec_dim])
            decoder_output_tensor = tflearn.lstm(first_dec_input, self.word_vec_dim, initial_state=states, return_seq=False, reuse=False, scope='decoder_lstm')
            decoder_output_sequence_single = tf.stack([decoder_output_tensor], axis=1)
            decoder_output_sequence_list = [decoder_output_tensor]
            # 再用解码器的输出作为下一个时序的输入
            for i in range(self.max_seq_len-1):
                if feed_previous:
                    next_dec_input = decoder_output_sequence_single
                else:
                    next_dec_input = tf.slice(decoder_inputs, [0, i+1, 0], [-1, 1, self.word_vec_dim])
                decoder_output_tensor = tflearn.lstm(next_dec_input, self.word_vec_dim, return_seq=False, reuse=True, scope='decoder_lstm')
                decoder_output_sequence_single = tf.stack([decoder_output_tensor], axis=1)
                decoder_output_sequence_list.append(decoder_output_tensor)
                
            
            decoder_output_sequence = tf.stack(decoder_output_sequence_list, axis=1)
           
            real_output_sequence = tf.concat(1, [encoder_output_sequence, decoder_output_sequence])
            #real_output_sequence = tf.stack([encoder_output_sequence,decoder_output_sequence],axis=2)
            
            
            
            #---change
            
           
            #targetY = tf.placeholder(shape=[None, max_seq_len,word_vec_dim], dtype=tf.float32, name="Y")
            
            net = tflearn.regression(real_output_sequence, optimizer='sgd', learning_rate=0.1, loss='mean_square', name="targetY")            
            net = tflearn.fully_connected(net, 1, activation='softmax')

            print "begin create DNN model"
            #model = tflearn.DNN(net)  
            model = tflearn.DNN(net, tensorboard_verbose=0, checkpoint_path=None)
            print "create DNN model finish"
            return model
    
    def train(self):
        trainXY, trainY = self.generate_trainig_data()
        model = self.model(feed_previous=False)
        model.fit(trainXY, trainY, n_epoch=1, snapshot_epoch=False, batch_size=1)
        model.save('/data/QA/model_tensorflow')
        return model
    
    def load(self):
        model = self.model(feed_previous=True,reuse=True)
        model.load('/data/QA/model_tensorflow')
        return model

In [57]:
my_seq2seq = MySeq2Seq( max_seq_len = 77, word_vec_dim = 100, input_file='/data/QA/QA_pair.txt')

In [58]:
tf.reset_default_graph()
model = my_seq2seq.train()

ValueError: Dimension 1 in both shapes must be equal, but are 1 and 77. Shapes are [?,1,100] and [?,77,100].
	From merging shape 0 with other shapes. for 'concat/concat_dim' (op: 'Pack') with input shapes: [?,1,100], [?,77,100].